In [1]:
#!pip install open3d
#!conda install -c open3d-admin open3d

In [2]:
import open3d as o3d
import numpy as np

ModuleNotFoundError: No module named 'open3d'

In [3]:
#pointCloudMatrix = [[0.0, 0, 0],[0, 0, 1],[0, 1, 0],[0, 1, 1],[1, 0, 0],[1, 0, 1],[1, 1, 0],[1, 1, 1]];

#pointCloudMatrix = np.array([[0,0,0]])
#for x in range(51):
   # for y in range(51):
       # for z in range(51):
          #  if (abs(x/50) == 1 or abs(y/50) == 1 or abs(z/50)==1 or abs(x) == 0 or abs(y) == 0 or abs(z)==0):
              #  pointCloudMatrix = np.concatenate((pointCloudMatrix,[[x/3,y/3,z/3]]),axis=0)
#pointCloudMatrix = np.array(np.loadtxt('np_array.txt'))                
pointCloudMatrix = np.array(np.loadtxt('decimated_hq_room_half.txt'))            


In [4]:
pointCloudMatrix.shape

(1013120, 8)

In [5]:
pointCloudMatrix= pointCloudMatrix[:,0:3]
pointCloudMatrix.shape

(1013120, 3)

In [18]:
def simplistic_mesh_creator_voxel(pointCloudMatrix):
     #takes a numpy matrix of size [x,3] of points 
     #and uses open3d inorder to create a basic voxel representation of the points
    
    o3dPointCloud = o3d.geometry.PointCloud() #Create pointcloud object
    o3dPointCloud.points = o3d.utility.Vector3dVector(pointCloudMatrix) #Populate with points from numpy matrix
    o3dPointCloud.scale(1 / np.max(o3dPointCloud.get_max_bound() - o3dPointCloud.get_min_bound()),
          center=o3dPointCloud.get_center()) #fix the scale
    #o3dPointCloud.colors = o3d.utility.Vector3dVector(np.random.uniform(0, 1, size=(N, 3)))
    #voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(o3dPointCloud,
                                                            #voxel_size=0.05)
        
    v_size = round(max(o3dPointCloud.get_max_bound()-o3dPointCloud.get_min_bound())*0.005,4) #find Side of each voxel
    voxel_grid=o3d.geometry.VoxelGrid.create_from_point_cloud(o3dPointCloud,voxel_size=v_size)
   # o3d.visualization.draw_geometries([voxel_grid]) #visualization
    
    voxels=voxel_grid.get_voxels()
    vox_mesh = o3d.geometry.TriangleMesh()
    #Generate Meshes from the voxels
    for v in voxels:
        cube = o3d.geometry.TriangleMesh.create_box(width=1, height=1,depth=1)
        cube.paint_uniform_color(v.color)
        cube.translate(v.grid_index, relative=False)
        vox_mesh+=cube
    
    vox_mesh.translate([0.5,0.5,0.5], relative=True)
    vox_mesh.scale(v_size, [0,0,0])
    vox_mesh.translate(voxel_grid.origin, relative=True)
    vox_mesh.merge_close_vertices(0.0000001) 
    o3d.visualization.draw_geometries([vox_mesh]) #visualization
    o3d.io.write_triangle_mesh("Voxel.obj", vox_mesh)
    
    #still need to export properly
        
    

In [19]:
simplistic_mesh_creator_voxel(pointCloudMatrix)

In [20]:
import matplotlib.pyplot as plt

In [21]:
def simplistic_mesh_creator_poisson(pointCloudMatrix):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pointCloudMatrix)
    pcd.scale(1 / np.max(pcd.get_max_bound() - pcd.get_min_bound()), center=pcd.get_center())
    #o3d.visualization.draw_geometries([pcd])

    pcd.normals = o3d.utility.Vector3dVector(np.zeros((1, 3))) #remove exiting Normals
    #pcd.estimate_normals() #Estimate new normals base on pointcloud
    radius = 0.1   # 
    max_nn = 30   # 
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius, max_nn))
    pcd.orient_normals_consistent_tangent_plane(100) #Takes a while but works!
    #pcd.orient_normals_consistent_tangent_plane(100) #Crashes program
    o3d.visualization.draw_geometries([pcd], point_show_normal=True)
    with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
            mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
                pcd, depth=9)
    o3d.visualization.draw_geometries([mesh])
    print('visualize densities')
    densities = np.asarray(densities)
    density_colors = plt.get_cmap('plasma')(
    (densities - densities.min()) / (densities.max() - densities.min()))
    density_colors = density_colors[:, :3]
    density_mesh = o3d.geometry.TriangleMesh()
    
    density_mesh.vertices = mesh.vertices
    density_mesh.triangles = mesh.triangles
    density_mesh.triangle_normals = mesh.triangle_normals
    density_mesh.vertex_colors = o3d.utility.Vector3dVector(density_colors)
    o3d.visualization.draw_geometries([density_mesh],mesh_show_back_face=True)
    
    o3d.io.write_triangle_mesh("density_mesh_possian_full.ply", density_mesh)
    
    
    print('remove low density vertices')

    #o3d.visualization.draw_geometries([density_mesh],mesh_show_back_face=True)
    vertices_to_remove = densities < np.quantile(densities, 0.05) #best Density For most cases
    density_mesh.remove_vertices_by_mask(vertices_to_remove)
    print(mesh)
    o3d.visualization.draw_geometries([density_mesh],mesh_show_back_face=True)
    o3d.io.write_triangle_mesh("density_mesh_possian_reduced.ply", density_mesh)
   

In [22]:
simplistic_mesh_creator_poisson(pointCloudMatrix)

[Open3D DEBUG] Input Points / Samples: 1013120 / 91088
[Open3D DEBUG] #   Got kernel density: 0.0999999 (s), 309.414 (MB) / 397.875 (MB) / 4229 (MB)
[Open3D DEBUG] #     Got normal field: 0.259 (s), 354.25 (MB) / 397.875 (MB) / 4229 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 1.415130e-06 / 1.433697e+00
[Open3D DEBUG] #       Finalized tree: 0.455 (s), 405.621 (MB) / 405.621 (MB) / 4229 (MB)
[Open3D DEBUG] #  Set FEM constraints: 1.069 (s), 385.727 (MB) / 405.621 (MB) / 4229 (MB)
[Open3D DEBUG] #Set point constraints: 0.208000 (s), 361.598 (MB) / 405.621 (MB) / 4229 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 1196630 / 1305200 / 62377
[Open3D DEBUG] Memory Usage: 361.602 MB
[Open3D DEBUG] # Linear system solved: 1.579 (s), 442.203 (MB) / 442.203 (MB) / 4229 (MB)
[Open3D DEBUG] Got average: 0.0320001 (s), 404.125 (MB) / 442.203 (MB) / 4229 (MB)
[Open3D DEBUG] Iso-Value: 4.993298e-01 = 5.058810e+05 / 1.013120e+06
[Open3D DEBUG] #          Total Solve:       7.0 (s

In [11]:
def generateConvexHull(pointCloudMatrix):

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pointCloudMatrix)
    pcd.scale(1 / np.max(pcd.get_max_bound() - pcd.get_min_bound()), center=pcd.get_center())

    hull, _ = pcd.compute_convex_hull()
    hull_ls = o3d.geometry.LineSet.create_from_triangle_mesh(hull)
    hull_ls.paint_uniform_color((0, 0, 1))
    o3d.visualization.draw_geometries([hull,hull_ls],mesh_show_back_face=True)
    o3d.io.write_triangle_mesh("ConvexHull.ply", hull)

In [12]:
generateConvexHull(pointCloudMatrix)

In [13]:
import os
import sys
import torch
!pip install pytorch3d
from pytorch3d.io import load_obj, save_obj
from pytorch3d.structures import Meshes
from pytorch3d.utils import ico_sphere
from pytorch3d.ops import sample_points_from_meshes,cubify
from pytorch3d.loss import (
    chamfer_distance, 
    mesh_edge_loss, 
    mesh_laplacian_smoothing, 
    mesh_normal_consistency,
    point_mesh_face_distance
)
import numpy as np
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
from tqdm.notebook import tqdm

In [14]:
# Set the device
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
    print("WARNING: CPU only, this will be slow!")

In [15]:

src_obj = os.path.join('voxel.obj')
verts, faces, aux = load_obj(src_obj)
faces_idx = faces.verts_idx.to(device)
verts = verts.to(device)
src_mesh = Meshes(verts=[verts], faces=[faces_idx])


trg_mesh = torch.tensor([pointCloudMatrix.tolist()])
target_point_count = trg_mesh.size(dim=1)

sample_trg = trg_mesh
sample_src = sample_points_from_meshes(src_mesh, 5000)


loss_chamfer, _ = chamfer_distance(sample_trg, sample_src)

print(loss_chamfer);

tensor(32.9979)
